In [ ]:
import requests
import pandas as pd

df = pd.read_parquet('data/selected_missing.parquet')

In [ ]:
import urllib.parse
import requests
from bs4 import BeautifulSoup
import json
import re

def write_cache(cache):
    with open('data/scraped.jsonl', 'a', encoding='utf-8') as file:
        for row in cache:
            file.write(json.dumps(row) + '\n')
    return []

def write_failures(failures):
    with open('data/failures.jsonl', 'a', encoding='utf-8') as file:
        for row in failures:
            file.write(json.dumps(row) + '\n')
    return []

errors = 0
cache = []
failures = []
i = 0
for idx, row in df.iterrows():
    if row['count'] > 117:
        continue
    i += 1
    if i % 100 == 0:
        print(i)
        print(f"failures: {len(failures)}")
        cache = write_cache(cache)
        failures = write_failures(failures)
    
    slug = '_'.join(row['lower_title'].split()).title()
    slug = urllib.parse.quote(slug, safe='')
    url = f"https://en.wikipedia.org/wiki/{slug}"

    try:
        r = requests.get(url)
        r.raise_for_status()
        errors = 0
    except Exception as e:
        print(e)
        failures.append({
            'lower_title': row['lower_title'],
            'count': row['count'],
            'url': url
        })
        errors += 1
        if errors > 10:
            raise Exception('Max errors exceeded.')
        continue

    cache.append({
        'lower_title': row['lower_title'],
        'count': row['count'],
        'url': url,
        'text': re.sub(r'\n+', '\n', BeautifulSoup(r.text).text)
    })

404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/United_States_Women%27S_National_Soccer_Team
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Cold_Semi-Arid_Climate
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Library_Of_Alexandria
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/People_%28American_Magazine%29
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/People%27S_Army_Of_Vietnam
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/King_Of_Portugal
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Games_%28Magazine%29
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Mathematical_Model
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Real_Line
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Frederick_Ii_Of_Prussia
404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/%C3%86thelred_The_Unread

KeyboardInterrupt: 